# Visualizações interativas (controles) - Bokeh

Este notebook apresenta os conceitos básicos de visualizações interativas ativas usando a biblioteca [Bokeh](https://bokeh.pydata.org). Estas visualizações permitem a inclusão de controles de interação com os dados na página renderizada pelo navegados.

Exemplos baseados no tutorial [Data Visualization with Bokeh in Python](https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-one-getting-started-a11655a467d4).

## Leitura e análise inicial dos dados


In [1]:
# Importando as bibliotecas
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()

%matplotlib inline

# lê o arquivo CSV
df = pd.read_csv('../data/aluguel.csv')

Loading BokehJS ...

Vamos utilizar um dataset de ofertas de aluguel. O primeiro passo ao se analisar dados desconhecidos é visualizar algumas linhas de dados:

In [2]:
df.head(10)

,codigo,endereco,quartos,suite,area,vaga,aluguel,condominio,data
0,34,Rua Desembargador Westphalen,2,0,90,0,900,371,11/10/17
1,167,Rua Jose Loureiro,2,0,64,0,650,428,15/07/17
2,6784,Rua Jose Loureiro,2,0,81,0,1100,400,23/08/17
3,82,Rua Lourenço Pinto,2,0,50,0,1350,300,19/09/17
4,2970,Rua Lourenço Pinto,2,0,63,0,1300,300,05/08/17
5,34197,Alameda Doutor Muricy,2,0,80,1,900,410,23/10/17
6,5072,Alameda Doutor Muricy,2,0,84,0,1100,382,02/09/17
7,469,Rua Desembargador Westphalen,1,0,30,0,550,210,03/07/17
8,24,Rua Desembargador Westphalen,1,0,60,1,800,120,30/09/17
9,74,Avenida Visconde de Guarapuava,2,1,132,1,1800,520,12/10/17


## Visualização com o Bokeh

Visualizar as distribuições das variaveis nos ajuda a identificar os primeiros padrões. Abaixo exibimos os histogramas para `aluguel` e `condomínio`. É possivel ver que a maior concentração de apartamentos é de aluguéis entre 600 e 900. Há também uma concentração menor em aluguéis em torno de 1200. Já a distribuição dos condomínios é mais homogênea, com a maior parte dos valores por volta de 370.

Faz sentido este comportamento? Aparentemente o condomínio não varia na mesma proporção do valor do aluguel. Ou seja, se um apartamento de 600 paga 300 de condomínio, não se espera que um apartamento de 1200 pague 600 de condomínio. Esta pode ser uma questão a se explorar em passos futuros das análises.

## Interatividade ativa no Bokeh

Nosso objetivo nos exemplos abaixo é usar controles de interface para modificar os dados do gráfico sendo plotado. Por exemplo, podemos usar CheckBoxes como o exibido abaixo para especificar o número de quartos desejado e filtrar os apartamentos exibidos.

In [3]:
# Contrói lista de possibilidades de números de quartos no dataset (neste caso apenas 1 e 2 quartos)
quartos_registrados = list(df['quartos'].unique().astype(str))
quartos_registrados.sort() # ordena a lista

# Cria o CheckBox contendo os valores da lista
selecao_quartos = CheckboxGroup(labels=quartos_registrados, active = [0, 1])

# Exibe o controle
show(selecao_quartos)

## Função de exibição do gráfico

Vamos definir uma função que exibe nosso gráfico baseado nos dados passados como parâmetro. Esta função será usada mais adiante quando formos unir todas as peças necessárias para a interação.

In [4]:
def make_plot(src):
    # Cria a figura
    p = figure(plot_width = 600, plot_height = 600, 
               title = 'Ofertas de Aluguel',
               x_axis_label = 'Área', y_axis_label = 'Aluguel')

    # Adiciona círgulos glyph
    p.circle('area', 'aluguel', source = src, size = 15, color = 'navy', hover_fill_color = 'red', alpha = 0.6, hover_fill_alpha = 1.0)

    # Adiciona texto que é mostrado quando se passa o mouse
    hover = HoverTool(tooltips = [('Condomínio', '@condominio'),
                                 ('Num quartos', '@quartos')])
    # Adiciona a funcionalidade de hover tool à figura
    p.add_tools(hover)

    return p

# Converte o DataFrame original para ser usado na plotagem
src = ColumnDataSource(df)

# Mostra a figura para testar
show(make_plot(df))

## Função de construção da fonte de dados

Precisamos também criar uma função para construir e reconstruir a fonte de dados à medida que alteramos os valores nos controles da interface. Como neste caso vamos adicionar um controle para o número de quartos, a função de construção deve receber uma lista com os números de quartos escolhidos. 

In [5]:
def make_dataset(quartos_plotar):
    # Filtra apenas os números de quartos escolhidos no DataFrame
    ofertas = df[df['quartos'].isin(quartos_plotar)]
    
    # Retornando o DatFrame construído
    return ofertas

Podemos testar a função para ver se está filtrando corretamente. Abaixo testamos a construção passando uma lista contendo apenas o elemento '1' indicando apenas apartamentos de 1 quarto:

In [6]:
df_ofertas = make_dataset(['1'])
df_ofertas

,codigo,endereco,quartos,suite,area,vaga,aluguel,condominio,data
7,469,Rua Desembargador Westphalen,1,0,30,0,550,210,03/07/17
8,24,Rua Desembargador Westphalen,1,0,60,1,800,120,30/09/17
10,9850,Avenida Visconde de Guarapuava,1,0,64,1,600,326,15/07/17
11,82343,Avenida Sete de Setembro,1,0,45,0,750,420,11/10/17
12,20802,Avenida Sete de Setembro,1,0,47,0,600,405,27/07/17
13,568,Rua Alferes Poli,1,0,43,0,600,330,12/08/17
14,294579,Avenida Silva Jardim,1,0,36,0,550,350,01/10/17
15,59375,Avenida Silva Jardim,1,0,33,0,560,305,11/09/17
16,80,Rua Desembargador Westphalen,1,0,80,1,900,350,12/08/17
17,66490,Rua Desembargador Westphalen,1,0,80,1,1100,350,29/08/17


## Função de atualização

A última função necessária é a `update`, que reage à manipulação dos controles e altera a fonte de dados usando a função definida acima. 

In [7]:
# Parâmetros default da função update com informações sobre o controle usado
def update(attr, old, new):
    # Obtém a lista de números de quartos selecionados nos CheckBoxes
    quartos_plotar = [selecao_quartos.labels[i] for i in 
                        selecao_quartos.active]

    # Cria nova fonte de dados baseada nas opções escolhidas
    new_src = make_dataset(quartos_plotar)
    
    # Converte o dataframe em ColumnDataSource
    new_src = ColumnDataSource(new_src)

    # Atualiza a fonte usada pelos glpyhs
    src.data.update(new_src.data)

## Juntando todas as peças

A função abaixo agrega todas as funções discutidas e inicializa os controles.

In [8]:
from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs


def modify_doc(doc):
    def make_dataset(quartos_plotar):
        ofertas = df[df['quartos'].isin(quartos_plotar)]
        return ofertas
    
    def update(attr, old, new):     
        quartos_plotar = [selecao_quartos.labels[i] for i in 
                            selecao_quartos.active]

        new_src = make_dataset(quartos_plotar)
        new_src = ColumnDataSource(new_src)

        src.data.update(new_src.data)
        
    def make_plot(src):
        p = figure(plot_width = 600, plot_height = 600, 
                   title = 'Ofertas de Aluguel',
                   x_axis_label = 'Área', y_axis_label = 'Aluguel')

        p.circle('area', 'aluguel', source = src, size = 15, color = 'navy', hover_fill_color = 'red', alpha = 0.6, hover_fill_alpha = 1.0)

        hover = HoverTool(tooltips = [('Condomínio', '@condominio'),
                                     ('Num quartos', '@quartos')])
        p.add_tools(hover)
        return p

    # Cria o controle de Check Boxes para o número de quartos
    selecao_quartos = CheckboxGroup(labels=quartos_registrados, active = [0, 1])
    
    # Conecta as mudanças nos checkboxes à função update
    selecao_quartos.on_change('active', update)
    
    # Adiciona os controles a um contêiner
    controls = WidgetBox(selecao_quartos)

    # Configuração e inicialização da fonte de dados (src) e do gráfico (p)
    quartos_plotar = [selecao_quartos.labels[i] for i in 
                            selecao_quartos.active]
    src = make_dataset(quartos_plotar)
    src = ColumnDataSource(src)

    p = make_plot(src)

    # Adiciona os controles e o gráfico a um elemento de layout
    layout = row(controls, p)

    # Adiciona o layout ao documento html
    doc.add_root(layout)
    

# Configura a aplicação
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [9]:
# Exibe a aplicação
show(app)

## Adicionando mais controles

No exemplo abaixo adicionamos mais um controlador para selecionar a faixa de valores de aluguel desejada.

In [10]:
def modify_doc(doc):
    def make_dataset(quartos_plotar, min_aluguel = 100, max_aluguel = 2000):
        ofertas = df[df['quartos'].isin(quartos_plotar) & (df['aluguel'] >= min_aluguel) & (df['aluguel'] <= max_aluguel)]
        return ColumnDataSource(ofertas)
    
    def update(attr, old, new):
        quartos_plotar = [selecao_quartos.labels[i] for i in 
                            selecao_quartos.active]

        new_src = make_dataset(quartos_plotar,
                               min_aluguel = selecao_aluguel.value[0],
                               max_aluguel = selecao_aluguel.value[1])

        src.data.update(new_src.data)
        
    def make_plot(src):
        p = figure(plot_width = 600, plot_height = 600, 
                   title = 'Ofertas de Aluguel',
                   x_axis_label = 'Área', y_axis_label = 'Aluguel')
        p.circle('area', 'aluguel', source = src, size = 15, color = 'navy', hover_fill_color = 'red', alpha = 0.6, hover_fill_alpha = 1.0)

        hover = HoverTool(tooltips = [('Condomínio', '@condominio'),
                                     ('Num quartos', '@quartos')])
        p.add_tools(hover)
        return p

    selecao_quartos = CheckboxGroup(name = 'Número de quartos', labels=quartos_registrados, active = [0, 1])
    
    selecao_quartos.on_change('active', update)
    

    # Inicializa slider para seleção de faixas de aluguel
    selecao_aluguel = RangeSlider(start = 100, end = 5000, value = (100, 2000),
                               step = 5, title = 'Faixa de aluguel')
    selecao_aluguel.on_change('value', update)
    
    controls = WidgetBox(selecao_quartos, selecao_aluguel)

    quartos_plotar = [selecao_quartos.labels[i] for i in 
                            selecao_quartos.active]

    src = make_dataset(quartos_plotar)

    p = make_plot(src)

    layout = row(controls, p)

    doc.add_root(layout)

handler = FunctionHandler(modify_doc)
app = Application(handler)

In [11]:
show(app)